In [3]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from msrest.authentication import CognitiveServicesCredentials

In [4]:


# Replace with your actual endpoint and subscription key
AZURE_ENDPOINT = "https://mytaxparser42ocr.cognitiveservices.azure.com/"
AZURE_KEY = "4hgTMwOZPX27iMGsYNDvNJpqPuOlvrTlzf5UxvS6zvIN1SnNarLwJQQJ99BBACYeBjFXJ3w3AAAFACOGKsyj"

# Initialize Computer Vision Client
client = ComputerVisionClient(AZURE_ENDPOINT, CognitiveServicesCredentials(AZURE_KEY))


In [5]:

with open(r"E:\Projects\OCR\ocr_testing\samples\color_images\Screenshot_20250211-234427_1.png", "rb") as image_file:
    response = client.read_in_stream(image_file, raw=True)








In [6]:
operation_location = response.headers["Operation-Location"]
operation_id = operation_location.split("/")[-1]
operation_id

'5c278a23-1ca2-4950-bf92-3610bc7207e1'

In [7]:
result = client.get_read_result(operation_id)


In [7]:
result.status


<OperationStatusCodes.succeeded: 'succeeded'>

In [8]:
result.as_dict()

{'status': 'succeeded',
 'created_date_time': '2025-02-28T18:18:33Z',
 'last_updated_date_time': '2025-02-28T18:18:34Z',
 'analyze_result': {'version': '3.2.0',
  'model_version': '2022-04-30',
  'read_results': [{'page': 1,
    'angle': 0.1802,
    'width': 551.0,
    'height': 986.0,
    'unit': 'pixel',
    'lines': [{'bounding_box': [21.0,
       12.0,
       172.0,
       12.0,
       172.0,
       29.0,
       21.0,
       29.0],
      'appearance': {'style': {'name': 'other', 'confidence': 0.972}},
      'text': 'Invoice Number:',
      'words': [{'bounding_box': [22.0,
         12.0,
         85.0,
         12.0,
         85.0,
         29.0,
         21.0,
         30.0],
        'text': 'Invoice',
        'confidence': 0.975},
       {'bounding_box': [91.0, 12.0, 172.0, 13.0, 172.0, 30.0, 90.0, 29.0],
        'text': 'Number:',
        'confidence': 0.994}]},
     {'bounding_box': [21.0, 47.0, 226.0, 47.0, 226.0, 65.0, 21.0, 66.0],
      'appearance': {'style': {'name': 'othe

{}

In [17]:
for x in result.analyze_result.read_results[0].lines:
    print(x.text)
    print(x.bounding_box)

Invoice Number:
[21.0, 12.0, 172.0, 12.0, 172.0, 29.0, 21.0, 29.0]
17256224121322859780
[21.0, 47.0, 226.0, 47.0, 226.0, 65.0, 21.0, 66.0]
Total Sales Value:
[23.0, 94.0, 183.0, 94.0, 183.0, 112.0, 23.0, 112.0]
3333.91
[21.0, 130.0, 87.0, 131.0, 87.0, 148.0, 21.0, 148.0]
Total Quantity:
[23.0, 176.0, 155.0, 177.0, 155.0, 198.0, 23.0, 197.0]
9.00
[23.0, 214.0, 61.0, 214.0, 62.0, 231.0, 23.0, 230.0]
Total Tax Charged:
[21.0, 259.0, 195.0, 260.0, 194.0, 280.0, 21.0, 279.0]
600.09
[23.0, 296.0, 84.0, 296.0, 84.0, 313.0, 23.0, 313.0]
Discount:
[22.0, 342.0, 108.0, 342.0, 108.0, 360.0, 22.0, 360.0]
0.00
[22.0, 379.0, 61.0, 379.0, 61.0, 396.0, 22.0, 395.0]
Total Bill Amount:
[22.0, 425.0, 181.0, 425.0, 181.0, 443.0, 22.0, 442.0]
3935.62
[21.0, 462.0, 95.0, 462.0, 95.0, 479.0, 22.0, 479.0]
Date Time:
[22.0, 508.0, 122.0, 508.0, 122.0, 526.0, 22.0, 525.0]
2024-12-13T22:05:00+05:00
[22.0, 544.0, 262.0, 544.0, 262.0, 562.0, 22.0, 562.0]
NTN:
[22.0, 592.0, 61.0, 590.0, 62.0, 607.0, 22.0, 607.0]
80

In [16]:
for x in result.analyze_result.read_results[0].lines:
    for y in x.words:
        print(y.text)

Invoice
Number:
17256224121322859780
Total
Sales
Value:
3333.91
Total
Quantity:
9.00
Total
Tax
Charged:
600.09
Discount:
0.00
Total
Bill
Amount:
3935.62
Date
Time:
2024-12-13T22:05:00+05:00
NTN:
8009003-2
Business
Name:
C
PLUS
SUPER
MARKET
Branch
Name:
C
PLUS
SUPER
MARKET
(UP
Morr)
Branch
Address:
Plot
A-01
(ST-09)
sector
11-1
north
karachi
POS
Counter:
172562


In [9]:
def extract_text_from_new_format(data):
    extracted_data_lst = []
    
    for page in data.get('analyze_result', {}).get('read_results', []):
        for line in page.get('lines', []):
            item = {}
            item['bounding_box'] = [
                [line['bounding_box'][0], line['bounding_box'][1]],
                [line['bounding_box'][2], line['bounding_box'][3]],
                [line['bounding_box'][4], line['bounding_box'][5]],
                [line['bounding_box'][6], line['bounding_box'][7]]
            ]
            item['text'] = line['text']
            item['confidence'] = min(word['confidence'] for word in line.get('words', [])) if line.get('words') else 1.0
            extracted_data_lst.append(item)
    
    return extracted_data_lst

# Example usage
new_texts = extract_text_from_new_format(result.as_dict())
print(new_texts)


[{'bounding_box': [[21.0, 12.0], [172.0, 12.0], [172.0, 29.0], [21.0, 29.0]], 'text': 'Invoice Number:', 'confidence': 0.975}, {'bounding_box': [[21.0, 47.0], [226.0, 47.0], [226.0, 65.0], [21.0, 66.0]], 'text': '17256224121322859780', 'confidence': 0.978}, {'bounding_box': [[23.0, 94.0], [183.0, 94.0], [183.0, 112.0], [23.0, 112.0]], 'text': 'Total Sales Value:', 'confidence': 0.994}, {'bounding_box': [[21.0, 130.0], [87.0, 131.0], [87.0, 148.0], [21.0, 148.0]], 'text': '3333.91', 'confidence': 0.995}, {'bounding_box': [[23.0, 176.0], [155.0, 177.0], [155.0, 198.0], [23.0, 197.0]], 'text': 'Total Quantity:', 'confidence': 0.994}, {'bounding_box': [[23.0, 214.0], [61.0, 214.0], [62.0, 231.0], [23.0, 230.0]], 'text': '9.00', 'confidence': 0.993}, {'bounding_box': [[21.0, 259.0], [195.0, 260.0], [194.0, 280.0], [21.0, 279.0]], 'text': 'Total Tax Charged:', 'confidence': 0.994}, {'bounding_box': [[23.0, 296.0], [84.0, 296.0], [84.0, 313.0], [23.0, 313.0]], 'text': '600.09', 'confidence': 